#### Load GPT 2 Model
GPT2 is a small model that can be used for small real time tasks where speed of the model is very important

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 75.6kB/s]


In [3]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [4]:
prompt = "The quick brown fox jumped over the "
inputs = tokenizer(prompt, return_tensors="pt")
inputs

{'input_ids': tensor([[  464,  2068,  7586, 21831, 11687,   625,   262,   220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
print(logits)

tensor([[[-36.2872, -35.0111, -38.0791,  ..., -40.5161, -41.3757, -34.9191],
         [-62.3139, -61.5645, -66.4938,  ..., -68.1284, -68.3227, -63.5829],
         [-66.3240, -66.7452, -72.1619,  ..., -75.1955, -73.4651, -68.1786],
         ...,
         [-79.0953, -81.2003, -86.2658,  ..., -90.5053, -87.8184, -83.2594],
         [-92.9352, -92.3160, -96.0634,  ..., -95.3959, -96.1432, -94.6931],
         [-63.6854, -65.3873, -66.4793,  ..., -73.0641, -69.6714, -67.3537]]])
